In [114]:
import numpy as np
import pandas as pd

from library import INITIAL_PERMUTATION_TABLE, INVERSE_PERMUTATION_TABLE, PC1_TABLE, PC2_TABLE, EXPANSION_TABLE, SBOX, PERMUTATION_TABLE

import textwrap

### Açık metine karşılık gelen ikili kod

In [115]:
def string_to_binary(plaintext):
    binary_plaintext = ''.join(format(ord(i),'b').zfill(8) for i in plaintext) 
    return binary_plaintext

### IP uygulanması

![IP](http://www.umsl.edu/~siegelj/information_theory/projects/des.netau.net/images/ip2.png)

In [116]:
def apply_initial_permutation(binary_plaintext,TABLE=INITIAL_PERMUTATION_TABLE):
    permutated_text = ""
    for index in TABLE:
        permutated_text += binary_plaintext[index-1]
    return permutated_text

### L0 ve R0 ayrımı

In [163]:
# def split_in_half(binarybits):
#     L0 = binarybits[:4]
#     R0 = binarybits[4:]
#     return L0,R0

### IP^-1 uygulanması

In [118]:
def apply_inverse_permutation(binary_plaintext,TABLE=INVERSE_PERMUTATION_TABLE):
    permutated_text = ""
    for index in TABLE:
        permutated_text += binary_plaintext[index-1]
    return permutated_text

## F FONKSİYONU

#### R0'a Expansion Table uygulanması

![](https://www.tutorialspoint.com/cryptography/images/des_specification.jpg)

In [119]:
def apply_expansion(R0,TABLE=EXPANSION_TABLE):
    bits48 = ""
    for index in TABLE:
        bits48 += R0[index-1]
    return bits48

#### XOR (K1 ile)

In [120]:
def XOR(bits1,bits2):
	xor_result = ""
	for index in range(len(bits1)):
		if bits1[index] == bits2[index]: 
			xor_result += '0'
		else:
			xor_result += '1'
	return xor_result

#### SBOX

![](https://www.researchgate.net/profile/Miroslaw-Szaban/publication/220440933/figure/fig7/AS:668741075484682@1536451705920/Function-S-box-S1-in-the-DES-algorithm-represented-as-a-table-and-its-work-on-the.ppm)

In [121]:
'''6 bitlik parçalara bölme'''

def split_in_6bits(XOR_48bits):
	list_of_6bits = textwrap.wrap(XOR_48bits,6)
	return list_of_6bits

In [122]:
'''İlk ve son bitin alınması'''

def get_first_and_last_bit(bits6):
	twobits = bits6[0] + bits6[-1] 
	return twobits

In [123]:
'''Ortadaki 4 bitin alınması'''

def get_middle_four_bit(bits6):
	fourbits = bits6[1:5] 
	return fourbits

In [124]:
'''İkili kodun 10luk sistemdeki karşılığı'''

def binary_to_decimal(binarybits):
	decimal = int(binarybits,2)
	return decimal

In [125]:
'''10luk sistemdeki sayının ikili karşılığı'''

def decimal_to_binary(decimal):
	binary4bits = bin(decimal)[2:].zfill(4)
	return binary4bits

In [139]:
'''Satır ve sütun numaralarından sbox karşılığı'''

def sbox_lookup(first_last,middle4):
	d_first_last = binary_to_decimal(first_last)
	d_middle = binary_to_decimal(middle4)
	
	sbox_value = SBOX[d_first_last][d_middle]
	return decimal_to_binary(sbox_value)

#### Permutation table

In [127]:
def apply_permutation(sbox_32bits,TABLE=PERMUTATION_TABLE):
	
	final_32bits = ""
	for index in TABLE:
		final_32bits += sbox_32bits[index-1]
	return final_32bits

### F fonksyonunun uygulanması

![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/Data_Encription_Standard_Flow_Diagram.svg/250px-Data_Encription_Standard_Flow_Diagram.svg.png)

In [153]:
bits6list = split_in_6bits("101110101010101011101101100110101010101101011111")
bits6list

['101110',
 '101010',
 '101011',
 '101101',
 '100110',
 '101010',
 '101101',
 '011111']

In [141]:
def functionF(pre32bits, key48bits):	
	result = ""
	expanded_right_half = apply_expansion(pre32bits)
	xor_value = XOR(expanded_right_half,key48bits)
	bits6list = split_in_6bits(xor_value)
	for bits6 in enumerate(bits6list):
		first_last = get_first_and_last_bit(bits6)
		middle4 = get_middle_four_bit(bits6)
		sboxvalue = sbox_lookup(first_last,middle4)
		result += sboxvalue
	final32bits = apply_permutation(result)	
	return final32bits

### ANAHTAR OLUŞTURMA

#### PC1

![PC!](http://www.umsl.edu/~siegelj/information_theory/projects/des.netau.net/images/PC-1.png)

In [129]:
def apply_pc1(key_64bits,TABLE = PC1_TABLE):
    key_56bits = ""
    for index in TABLE:
        key_56bits += key_64bits[index-1]
    return key_56bits

#### Left Key ve Right Key ayrımı

In [130]:
def split_in_half(key_56bits):
    left_key = key_56bits[:28]
    right_key = key_56bits[28:]
    return left_key,right_key

#### Circular Left Shift

![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/Rotate_left.svg/300px-Rotate_left.svg.png)

In [131]:
def circular_left_shift(left_key,right_key):
    shifted_left = left_key[1:] + left_key[:1]
    shifted_right = right_key[1:] + right_key[:1]
    shifted = shifted_left + shifted_right
    return shifted

#### PC2

![](http://www.umsl.edu/~siegelj/information_theory/projects/des.netau.net/images/permuted%20choice2.png)

In [137]:
def apply_pc2(shifted,TABLE=PC2_TABLE):
    key_48bits = ""
    for index in TABLE:
        key_48bits += shifted[index-1]
    return key_48bits

### Anahtarın oluşturulması

![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/06/DES-key-schedule.png/250px-DES-key-schedule.png)

In [133]:
def generate_key(key_64bits):
    pc1_out = apply_pc1(key_64bits) 
    C0,D0 = split_in_half(pc1_out)
    shifted = circular_left_shift(C0,D0)
    key = apply_pc2(shifted)
    return key

# DES ALGORİTMASI

![](https://www.researchgate.net/profile/Adeolu-Afolabi/publication/306425963/figure/fig2/AS:398613525090307@1472048278894/Depiction-of-One-Round-of-DES-212-3-DES-Algorithm-In-cryptography-techniques-Triple.png)

In [157]:
def DES_encrypt(message,key):
	cipher = ""

	plaintext_bits = string_to_binary(message)
	key_bits = string_to_binary(key)
	
	roundkey = generate_key(key_bits)
	
	
	
	p_plaintext = apply_initial_permutation(plaintext_bits)
	
	L0,R0 = split_in_half(p_plaintext)

	R1 = XOR(L0,functionF(R0, roundkey))
	L1 = R0

	cipher = apply_inverse_permutation(L1+R1)
	
	return cipher

In [158]:
cipher = DES_encrypt("kemaloze", "21721149")
cipher

'1001110010111111100111110011000011100111011011000100010101100100'